In [3]:
import pandas as pd
import numpy as np
import json

from slide_windows import condition

# 假设你的数据已经被加载到一个 DataFrame 中

df = pd.read_csv('Final_table.csv')

# 创建一个以 NOC 为索引的 DataFrame，并获取每个 NOC 对应的参赛年份
noc_years = df.groupby('NOC')['Year'].apply(list).reset_index()
# print(len(noc_years))
# 输出结果
windows = {}
def clean_list(mixed_list):
    int_list = []  # 创建一个新的列表用于存储转换后的整数
    for item in mixed_list:
        try:
            int_value = int(item)  # 尝试将每个元素转换为整数
            int_list.append(int_value)  # 添加到新列表中
        except (ValueError, TypeError):
            # 如果转换失败，跳过该项
            continue
    return int_list

In [8]:
noc_years

,NOC,Year
0,AFG,"[2008, 2012]"
1,ALB,[2024]
2,ARE,"[2004, 2016]"
3,ARG,"[1924, 1928, 1932, 1936, 1948, 1952, 1956, 196..."
4,ARM,"[1996, 2000, 2008, 2012, 2016, 2020, 2024]"
...,...,...
137,WSM,[2008]
138,YUG,"[1924, 1928, 1936, 1948, 1952, 1956, 1960, 196..."
139,ZAF,"[1908, 1912, 1920, 1924, 1928, 1932, 1936, 194..."
140,ZMB,"[1984, 1996, 2024]"


In [23]:
data = {}
for i in range(len(noc_years)):
    noc = noc_years.iloc[i]['NOC']
    year = noc_years.iloc[i]['Year']
    data[noc] = []
    order = 0
    # print(noc_years[])
    if len(year) < 3:
        continue
    for j in range(3,len(year)):
        tmp = {}
        tmp['Year'] = year[j]
        tmp['Timeoder'] = order
        order+=1
        tmp['Feats'] = []
        for k in range(j-3,j):
            condition = (df['Year'] == year[k]) & (df['NOC'] == noc)
            find = df[condition]
            row = find.iloc[0].drop(['Year', 'NOC', 'Rank']).tolist()
            row = clean_list(row)
            tmp['Feats'].extend(row)
        condition = (df['NOC'] == noc) & (df['Year'] == year[j])
        find = df[condition]
        host_flag = find.iloc[0]['host_flag']
        total = find.iloc[0]['Total']
        tmp['Feats'].append(int(host_flag))
        tmp['Label'] = int(total)
        data[noc].append(tmp)

In [24]:
print(data)

{'AFG': [], 'ALB': [], 'ARE': [], 'ARG': [{'Year': 1936, 'Timeoder': 0, 'Feats': [1, 3, 2, 6, 109, 0, 3, 3, 1, 7, 98, 0, 3, 1, 0, 4, 47, 0, 0], 'Label': 7}, {'Year': 1948, 'Timeoder': 1, 'Feats': [3, 3, 1, 7, 98, 0, 3, 1, 0, 4, 47, 0, 2, 2, 3, 7, 68, 0, 0], 'Label': 7}, {'Year': 1952, 'Timeoder': 2, 'Feats': [3, 1, 0, 4, 47, 0, 2, 2, 3, 7, 68, 0, 3, 3, 1, 7, 295, 0, 0], 'Label': 5}, {'Year': 1956, 'Timeoder': 3, 'Feats': [2, 2, 3, 7, 68, 0, 3, 3, 1, 7, 295, 0, 1, 2, 2, 5, 174, 0, 0], 'Label': 2}, {'Year': 1960, 'Timeoder': 4, 'Feats': [3, 3, 1, 7, 295, 0, 1, 2, 2, 5, 174, 0, 0, 1, 1, 2, 44, 0, 0], 'Label': 2}, {'Year': 1964, 'Timeoder': 5, 'Feats': [1, 2, 2, 5, 174, 0, 0, 1, 1, 2, 44, 0, 0, 1, 1, 2, 116, 0, 0], 'Label': 1}, {'Year': 1968, 'Timeoder': 6, 'Feats': [0, 1, 1, 2, 44, 0, 0, 1, 1, 2, 116, 0, 0, 1, 0, 1, 148, 0, 0], 'Label': 2}, {'Year': 1972, 'Timeoder': 7, 'Feats': [0, 1, 1, 2, 116, 0, 0, 1, 0, 1, 148, 0, 0, 0, 2, 2, 132, 0, 0], 'Label': 1}, {'Year': 1988, 'Timeoder': 8, 'Fe

In [25]:
with open("new_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4)